In [96]:
import os
import datasets
import transformers
from huggingface_hub import notebook_login
from dotenv import load_dotenv
load_dotenv(verbose=True)
import torch
import wandb

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [99]:
# device
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

False

In [3]:
seed = 42
wandb.init(project='kobart')

# Loading dataset

In [4]:
api_token = os.getenv('HF_DATASET_API_TOKEN')
dataset = datasets.load_dataset('metamong1/summarization_paper', use_auth_token=api_token)

Reusing dataset paper_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 73640
    })
    validation: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 18411
    })
})

## Checking dataset content

In [6]:
isinstance(dataset, datasets.dataset_dict.DatasetDict) 

True

In [7]:
type(dataset['train'])

datasets.arrow_dataset.Dataset

In [8]:
def show_samples(dataset, num_samples=3, seed=seed):
    if isinstance(dataset, datasets.dataset_dict.DatasetDict):
        sample = dataset['train'].shuffle(seed=seed).select(range(num_samples))
    elif isinstance(dataset, datasets.arrow_dataset.Dataset):
        sample = dataset.shuffle(seed=seed).select(range(num_samples))
    else:
        raise ValueError('Inappropriate dataset.')
    for example in sample:
        print(f"\n'>> Title: {example['title']}'")
        print(f"'>> Text: {example['text']}'")

In [9]:
dataset['train'][0]

{'doc_id': 'A201008192682',
 'title': '횡적등방성 매질에서 중합전 역시간 구조보정',
 'text': '역시간 구조보정은 음원영역 파동장 외삽과 수진기영역 파동장 외삽의 상호상관으로 지층구조를 영상화하는 방법으로 복잡한 등방성 매질 층서구조를 영상화하는데 주로 이용된다. 그러나 일반적으로 지구내부 지층구조는 이방성 특성을 지니고 있으므로 이방성을 고려한 구조보정 기술이 필요하다. 여기에서는 편미분 파동장과 음원모음의 내적에 의한 알고리즘과 가상음원과 역전파 파동장과의 내적에 의한 알고리즘을 이용하여 횡적등방성 매질에서 역시간 구조보정 기술을 개발하고자 하였다. 단순 이방성 지층모델에 대한 수치모형실험 결과, 두 가지 방법에 의한 지층단면도 영상은 거의 차이가 없어 가상음원과 역전파 파동장과의 내적으로 구조보정을 실시하는 것이 효과적임을 알 수 있었다. 수평적으로 속도가 변하는 이방성 매질 지층구조에서 편미분 파동장을 구하지 않고 영상화 할 수 있음을 알 수 있었다.',
 'doc_type': '논문',
 'file': '논문요약_0206_0.json'}

In [10]:
show_samples(dataset)

Loading cached shuffled indices for dataset at /opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c/cache-349ab71b5c228bee.arrow



'>> Title: 민주주의 관점으로 본 국가기록관리체계 평가와 전망'
'>> Text: 본 논문은 국가기록관리체계를 민주주의 관점에서 보았다. 지난 10년의 국가기록관리체계를 평가하고 새로운 국가기록관리체계를 전망하기위해서는 민주주의가 우선이 되어야 할 것 같다. 지난 10년의 아카이브가쭈그러진 아카이브였다면, 새로운 아카이브는 어떤 아카이브이어야 할까? 쭈그러진 깡통을 펴듯이 망가지기 전의 상태로 복구하는 게 필요한 일인줄 알면서도 새로운 기록풍경을 그리는 것도 그 못지않게 중요하다고 보았다. 본 논문은 기본적으로 민주주의의 가치를 제도화하는 아카이브 전망에대한 것이다. 나아가 일상적 민주주의에서 아카이브는 무엇을 할 수 있을지도 짚어보았다. 이를 위해 그간의 국가기록관리기구 개편 제안을 체계적으로 검토했다. 그 다음에는 공공기록관 기록관리직의 목소리를 재현해보았다. 기록관리직은 지난 10년 동안 기록공동체와 우리 사회가 일군 사회적자산이기 때문에 그 목소리가 무엇을 원하는지 들어야 한다고 판단했기 때문이다. 기록은 이제 단순히 통치의 수단이 아니라 그 통치의 정통성을 규정하는 통치의 기반이 되었다. 이처럼 기록의 사회적 역할과 의미가 달라졌다. 1999년 시점이 아닌 2017년 시점에서, 국가아카이브의 기록풍경을 다시 그려야 할 것이다. 이런 과제를 위해서는 무엇보다도 민주주의 관점이 필요하다고 생각한다.'

'>> Title: 조선의 대일 교화 양상과 그 기저 (朝鮮의 對日 敎化 樣相과 그 基底)'
'>> Text: 조선전기는 왜구의 잦은 침탈로 인해 일본과의 외교적 해결이 절실히 요구되던 때였다. 그러나 막부의 왜구 통제력 부족으로 교섭이 번번이 무위로 돌아갔다. 그와 같은 상황이 지속되자 명쾌한 대책을 마련하지 못한 조선은 信義를 잃은 일본에 대하여 夷狄으로 대하거나 小國으로 여기는 입장을 취하였다. 이러한 당시의 대일 관계는 문인들에 의해 시문으로 구현되었다. 하지만 시문을 살펴보면, 현안인 왜구를 근절하는 데에만 집중한 나머지 조일 간의 다양한

# Training with sample dataset

### Dividing sample dataset

In [11]:
sample_size = 1000
sample_training_dataset = dataset['train'].shuffle(seed=seed).select(range(sample_size))

Loading cached shuffled indices for dataset at /opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c/cache-349ab71b5c228bee.arrow


In [12]:
show_samples(sample_training_dataset)

Loading cached shuffled indices for dataset at /opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c/cache-781b22210d45164a.arrow



'>> Title: 탄소라벨링 브랜드 충성도를 결정하는 요인: 가치-­태도­-행동 모형의 적용'
'>> Text: 기후변화와 온실가스 감축에 대한 사회적 관심과 정부의 정책이 증가하는 요즘 탄소 라벨링 제도는 저탄소 생산과 저탄소 소비를 연결하는 환경정책으로 시장에 점차 확대되고 있다. 따라서 탄소 라벨링 제품에 대한 소비자 태도와 브랜드 충성도를 분석하기 위하여 탄소 라벨링 소비자 모형을 제시하여, 소비자의 내재된 가치가 탄소 라벨링 제품 및 기업 이미지 형성에 영향을 주고 나아가 브랜드 충성도를 제고하는 과정을 분석하였다. 2차에 걸친 설문조사를 통해 패널 데이터를 수집하여 분석한 결과 소비자의 자율성 가치는 지각된 통제소재에 긍정적인 영향을 주고 기업 이미지를 긍정적으로 형성시켰으며, 환경적 가치는 지각된 소비자 효과를 높이고, 나아가 지각된 장애를 줄임으로써 제품 이미지에 영향을 미침을 확인하였다. 궁극적으로, 긍정적인 기업 이미지와 제품 이미지는 브랜드 충성도를 향상시켰다. 이와 같은 결과는 탄소 라벨링 정책이 기후변화 대응을 위해 온실가스를 감축하는 데 도움이 될 뿐만 아니라 동시에 소비자의 기업 및 제품에 대한 이미지와 브랜드 충성도를 향상시키는 순기능이 있음을 보여준다. 탄소 라벨링 정책이 소비자 태도와 브랜드 충성도에 미치는 영향을 분석하는 고유의 모형을 제시하고 실증분석한 점에 그 학문적 기여도가 높다고 하겠다. 더욱이, 연구결과는 정부에게 환경정책의 효율성을 높이기 위한 정책제언을 제시하고 있고, 기업에게도 탄소 라벨링과 관련된 마케팅 전략의 방향성을 제안하고 있다는 점에서 실무적 공헌을 갖고 있다.'

'>> Title: 3DTV 구매의도에 영향을 미치는 요인에 대한 연구'
'>> Text: 본 연구는 3DTV의 구매의도에 영향을 미치는 요인들을 인지적 요인과 감정적 요인으로 접근하였으며 긍정적 측면과 부정적 측면을 함께 고려하여 살펴보았다. 3DTV의 수용에 영향을 미치는 인지적 요인으로써‘지각된 유용성’과‘지각된 사용의 용이성’, 그

### Tokenizer

In [13]:
from transformers import AutoTokenizer

model_checkpoint = "gogamza/kobart-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/opt/conda/envs/final/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [14]:
inputs = tokenizer("제목 생성을 위한 요약 모델을 이제부터 만들어 봅시다!")
inputs

{'input_ids': [12147, 10608, 14106, 14403, 14353, 26200, 24224, 14667, 14150, 14803, 24110, 11465, 9754, 232], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['제',
 '목',
 '▁생',
 '성을',
 '▁위한',
 '▁요약',
 '▁모델을',
 '▁이제',
 '부터',
 '▁만들어',
 '▁봅',
 '시',
 '다',
 '!']

In [72]:
max_input_length = 512
max_target_length = 30

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples['text'], max_length=max_input_length, truncation = True, #padding=True
    )

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['title'], max_length=max_target_length, truncation=True, #padding=True
        )
    
    model_inputs['labels'] = labels['input_ids']

    return model_inputs

In [73]:
tokenized_datasets = sample_training_dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [74]:
tokenized_eval_datasets = dataset['validation'].select(range(500)).map(preprocess_function, batched=True)

[Korean Sentence Splitter]: Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c/cache-34efb215017ddbce.arrow


### Baseline score

In [75]:
# from datasets import load_metric
# rouge_score = load_metric('rouge')

import os, sys
sys.path.append('/opt/ml/final-project-level3-nlp-02')
from rouge import compute


In [76]:
generated_summary = "이번 제목 생성 테스크 진짜 진짜 잘하고 싶다. 두번째줄은 작동 안한다고?"
reference_summary = "이번 제복 생성 테스크 잘하고 싶다. 두번째 작동 안한다고?"
# generated_summary = "I absolutely loved reading the Her Games"
# reference_summary = "I loved reading the Hunger Games"ung
scores = compute(
    predictions=[generated_summary], references=[reference_summary], tokenizer=tokenizer
)
scores['rouge1'].mid

Score(precision=0.7368421052631579, recall=0.875, fmeasure=0.7999999999999999)

In [77]:
import kss
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

def one_sentence_title(text):
    return kss.split_sentences(text)[0]

In [78]:
print(one_sentence_title(sample_training_dataset['text'][0]))

본 논문은 국가기록관리체계를 민주주의 관점에서 보았다.


In [79]:
def evaluate_baseline(dataset, tokenizer):
    summaries = [one_sentence_title(text) for text in dataset['text']]
    return compute(predictions=summaries, references=dataset['title'], tokenizer=tokenizer)

In [80]:
import pandas as pd

score = evaluate_baseline(dataset['validation'].shuffle().select(range(10)), tokenizer)
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
rouge_dict = dict((rn, round(score[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 19.64, 'rouge2': 11.76, 'rougeL': 19.08, 'rougeLsum': 19.06}

## Fine Tuning

In [81]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

loading configuration file https://huggingface.co/gogamza/kobart-summarization/resolve/main/config.json from cache at /opt/ml/.cache/huggingface/transformers/1c32baaf6a1067a5e27a0dfbac0a3d23a86d958ab10b092d5ea4150bd451de17.4e52ef6c87e6938c92ba0d19888607d76e30e950e81060a8fa6cb1189c93614d
/opt/conda/envs/final/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Model config BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "classif_

In [82]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8

# Show the training loss with every epoch
logging_steps = len(tokenized_datasets) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f'{model_name}-finetuned-paper-sample-size-1000',
    evaluation_strategy='epoch',
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay = 0.01,
    save_total_limit=2,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    report_to='wandb'
    # push_to_hub=True,
)

PyTorch: setting up devices


In [83]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE expects a newline after each sentence
    # decoded_preds = ['\n'.join(kss.split_sentences(pred.strip())) for pred in decoded_preds]
    # decoded_labels = ['\n'.join(kss.split_sentences(label.strip())) for label in decoded_labels]

    # Compute ROUGE scores
    result = compute(
        predictions=decoded_preds, references=decoded_labels, tokenizer=tokenizer, use_stemmer=True
    )

    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [84]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [85]:
len(tokenized_datasets[5]['attention_mask'])

325

In [86]:
columns_to_return = ['input_ids', 'labels', 'attention_mask']
tokenized_datasets.set_format(type='torch', columns = columns_to_return, device=device)


In [87]:
features = [tokenized_datasets[i] for i in range(2)]
data_collator(features)

{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'input_ids': tensor([[10888, 14396, 19255,  ...,     3,     3,     3],
        [16446, 12126, 14575,  ...,  3020, 18037, 14098]]), 'labels': tensor([[15352, 14453, 14073, 16439, 14285, 14518, 17469, 15095, 20044, 14653,
         11863, 14718,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [16446, 12024, 14029, 12041, 14139, 13714, 14256, 17664, 14028, 14040,
         12123, 14338,  4543,  8813, 12024,  1700,  3486,  4400,  1700,  4336,
          2682,  1700,  4823,  5996,  9120, 14028,  1700,  3095,  3704,   240]]), 'decoder_input_ids': tensor([[    2, 15352, 14453, 14073, 16439, 14285, 14518, 17469, 15095, 20044,
         14653, 11863, 14718,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    2, 16446, 12024, 140

In [88]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_eval_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    
)

In [90]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: file, text, doc_type, doc_id, title.
***** Running training *****
  Num examples = 1000
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1000


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 